In [1]:
from ga4gh.vrs import models, vrs_deref, vrs_enref
from ga4gh.core import ga4gh_identify, ga4gh_serialize, ga4gh_digest, ga4gh_deref

import json
def ppo(o, indent=2):
    """pretty print object as json"""
    print(json.dumps(o.as_dict(), sort_keys=True, indent=indent))
    
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from biocommons.seqrepo import SeqRepo
from ga4gh.vrs.extras.translator import Translator

Removing allOf attribute from CopyNumber to avoid python-jsonschema-objects error.
Removing allOf attribute from SequenceInterval to avoid python-jsonschema-objects error.
Removing allOf attribute from RepeatedSequenceExpression to avoid python-jsonschema-objects error.
/Users/ahw001/.local/share/virtualenvs/vrs-python-z7zdIrLG/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [2]:
seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

tlr = Translator(data_proxy=dp)

## Representing ClinVar PGx Allele 634420
https://www.ncbi.nlm.nih.gov/clinvar/variation/634420

In [3]:
# "Haplotype" 1: https://www.ncbi.nlm.nih.gov/clinvar/variation/633851

In [4]:
hgvs_expression = "NC_000010.11:g.94761665G>A"

In [5]:
h1_allele1 = tlr.translate_from(hgvs_expression,'hgvs')

In [6]:
# Haplotype 2: https://www.ncbi.nlm.nih.gov/clinvar/variation/633860/
# H2 Allele 1: https://www.ncbi.nlm.nih.gov/clinvar/variation/16900
# H2 Allele 2: https://www.ncbi.nlm.nih.gov/clinvar/variation/39357

hgvs_expression = "NC_000010.11:g.94762706A>G"
h2_allele1 = tlr.translate_from(hgvs_expression,'hgvs')

hgvs_expression = "NC_000010.11:g.94761900C>T"
h2_allele2 = tlr.translate_from(hgvs_expression,'hgvs')

In [7]:
h2 = models.Haplotype(members=[h2_allele1, h2_allele2])

In [8]:
models.Number(value=1)

<Number type=<Literal<str> Number> value=<Literal<int> 1>>

In [10]:
gt_mem1 = models.GenotypeMember(copies=models.Number(value=1), variation=h1_allele1)

In [11]:
gt_mem2 = models.GenotypeMember(variation=h2, copies=models.Number(value=1))
gt = models.Genotype(members=[gt_mem1, gt_mem2], copies=models.Number(value=2))

In [12]:
ppo(gt)

{
  "copies": {
    "type": "Number",
    "value": 2
  },
  "members": [
    {
      "copies": {
        "type": "Number",
        "value": 1
      },
      "type": "GenotypeMember",
      "variation": {
        "_id": "ga4gh:VA.3AIu_q8I6dxuX76aZIq43GvogEsLuvAN",
        "location": {
          "interval": {
            "end": {
              "type": "Number",
              "value": 94761665
            },
            "start": {
              "type": "Number",
              "value": 94761664
            },
            "type": "SequenceInterval"
          },
          "sequence_id": "ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB",
          "type": "SequenceLocation"
        },
        "state": {
          "sequence": "A",
          "type": "LiteralSequenceExpression"
        },
        "type": "Allele"
      }
    },
    {
      "copies": {
        "type": "Number",
        "value": 1
      },
      "type": "GenotypeMember",
      "variation": {
        "members": [
          {
           

In [18]:
gt._id = ga4gh_identify(gt)

In [19]:
ppo(gt)

{
  "_id": "ga4gh:GT.W2sHtmm7mse3anXQ3KvlVk1TL-YeA37b",
  "copies": {
    "type": "Number",
    "value": 2
  },
  "members": [
    {
      "copies": {
        "type": "Number",
        "value": 1
      },
      "type": "GenotypeMember",
      "variation": {
        "_id": "ga4gh:VA.3AIu_q8I6dxuX76aZIq43GvogEsLuvAN",
        "location": {
          "interval": {
            "end": {
              "type": "Number",
              "value": 94761665
            },
            "start": {
              "type": "Number",
              "value": 94761664
            },
            "type": "SequenceInterval"
          },
          "sequence_id": "ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB",
          "type": "SequenceLocation"
        },
        "state": {
          "sequence": "A",
          "type": "LiteralSequenceExpression"
        },
        "type": "Allele"
      }
    },
    {
      "copies": {
        "type": "Number",
        "value": 1
      },
      "type": "GenotypeMember",
      "vari